# Сводная таблица признаков

Импортируем библиотеки и делаем некоторые настройки

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)# вывод всех колонок фрейма

Загружаем признаки для events_train.csv и events_data_test.csv

In [3]:
users_data = pd.read_csv('DATA\\feature_data_events_22.zip', compression='zip')

Загружаем признаки для submissions_train.csv и submission_data_test.csv

In [4]:
data = pd.read_csv('DATA\\feature_data_submissions_16.zip', compression='zip')

In [5]:
users_data = users_data.merge(data, on='user_id', how='outer')

In [6]:
# заполнил пропуски по разному
users_data['S_steps_ratio'] = users_data['S_steps_ratio'].fillna(1)   
users_data['c_freq'] = users_data['c_freq'].fillna(25418 - 12743 + 199)
users_data['c_density'] = users_data['c_density'].fillna((25418 - 12743 + 199) / 25418)
users_data['c_freq[i]/c_freq[i+1]'] = users_data['c_freq[i]/c_freq[i+1]'].fillna(12874 / 1181)

In [7]:
# остальное заполнил нулями
users_data = users_data.fillna(0)

Признаки-различные агрегации

In [8]:
#список признаков для агрегации
name = ['d_count', 'p_count', 's_count', 'v_count', 's_uniq', 'v_uniq',
        'c_count', 'w_count', 'c_uniq', 'w_uniq']

In [9]:
users_data['sum'] = users_data[name].sum(axis=1)
users_data['mean'] = users_data[name].mean(axis=1)
users_data['median'] = users_data[name].median(axis=1)
users_data['std'] = users_data[name].std(axis=1)
users_data['max'] = users_data[name].max(axis=1)
users_data['min'] = users_data[name].min(axis=1)

Отношения

In [10]:
users_data['d_count_ratio'] = users_data['d_count'] / users_data['sum']
users_data['p_count_ratio'] = users_data['p_count'] / users_data['sum']
users_data['s_count_ratio'] = users_data['s_count'] / users_data['sum']
users_data['v_count_ratio'] = users_data['v_count'] / users_data['sum']
users_data['s_uniq_ratio'] = users_data['s_uniq'] / users_data['sum']
users_data['v_uniq_ratio'] = users_data['v_uniq'] / users_data['sum']
users_data['c_count_ratio'] = users_data['c_count'] / users_data['sum']
users_data['w_count_ratio'] = users_data['w_count'] / users_data['sum']
users_data['c_uniq_ratio'] = users_data['c_uniq'] / users_data['sum']
users_data['w_uniq_ratio'] = users_data['w_uniq'] / users_data['sum']

Добавим признаки количество и распределение дней

In [11]:
data = pd.read_csv('DATA\\days_feature_target.csv')

In [12]:
users_data = users_data.merge(data, on='user_id', how='outer')

In [13]:
#лишнее удаляем
users_data = users_data.drop(['s_count', 'v_count', 'c_count', 'w_count'], axis=1)

Относительные разности и разности дат 

In [14]:
data = pd.read_csv('DATA\\daysdiff_ratio.csv')

In [15]:
users_data = users_data.merge(data, on='user_id', how='outer')

Временные активности

In [16]:
data = pd.read_csv('DATA\\week_weekday_day.zip', compression='zip')

In [17]:
users_data = users_data.merge(data, on='user_id', how='inner')

Переставим колонки, сгруппировав "однотипные" признаки

In [18]:
users_data = users_data[['user_id',
                         'c_uniq', 'w_uniq', 'd_count', 'p_count', 's_uniq', 'v_uniq', 
                         'c_diff', 'w_diff', 's_diff', 'v_diff',
                         'sum', 'mean', 'median', 'std', 'max', 'min', 'hours_sum',
                         'time_diff', 'sum_diff',
                         'events_day', 'submissions_day',
                         'steps_day', 'steps_night',
                         'p_count*steps_ratio', 'c_uniq*S_steps_ratio',
                         '(p_freq[i]/p_freq[i+1])*(p_count*steps_ratio)',
                         '(c_freq[i]/c_freq[i+1])*(c_uniq*S_steps_ratio)',
                         'p_freq', 'c_freq',
                         'c_div', 'w_div', 's_div', 'v_div',
                         'd_count_ratio', 'p_count_ratio', 's_count_ratio', 'v_count_ratio',
                         's_uniq_ratio', 'v_uniq_ratio',
                         'c_count_ratio', 'w_count_ratio', 'c_uniq_ratio', 'w_uniq_ratio',
                         'steps_ratio', 'S_steps_ratio',
                         'day_ratio', 'hours_ratio', 'time_ratio',
                         'p_freq[i]/p_freq[i+1]', 'c_freq[i]/c_freq[i+1]', 
                         'p_density', 'c_density',
                         'loc_day',
                         'koef_weekday_1', 'koef_weekday_2', 'koef_weekday_3',
                         'koef_day_1', 'koef_day_2', 'koef_day_3',
                         'koef_week_mean',
                         'end_wrong',  'end_correct']]

Разделим выборку на train и test

In [19]:
#загрузка целевого признака
target_feature = pd.read_csv('DATA\\target_feature.csv')

In [20]:
train_data = users_data.merge(target_feature, on='user_id', how='inner')

In [21]:
train_data = train_data.drop(['is_gone'], axis=1)

In [22]:
data = pd.read_csv('DATA\events_data_test.csv')

In [23]:
data = data.drop(['step_id', 'timestamp', 'action'], axis=1)

In [24]:
data = data.drop_duplicates(subset=['user_id'])

In [25]:
test_data = users_data.merge(data, on='user_id', how='inner')

Сохраним результат

In [26]:
train_data.to_csv('DATA\\train_data_19234_62.zip', index=False, compression='zip')

In [27]:
test_data.to_csv('DATA\\test_data_6184_62.zip', index=False, compression='zip')

In [28]:
display(test_data)

,user_id,c_uniq,w_uniq,d_count,p_count,s_uniq,v_uniq,c_diff,w_diff,s_diff,v_diff,sum,mean,median,std,max,min,hours_sum,time_diff,sum_diff,events_day,submissions_day,steps_day,steps_night,p_count*steps_ratio,c_uniq*S_steps_ratio,(p_freq[i]/p_freq[i+1])*(p_count*steps_ratio),(c_freq[i]/c_freq[i+1])*(c_uniq*S_steps_ratio),p_freq,c_freq,c_div,w_div,s_div,v_div,d_count_ratio,p_count_ratio,s_count_ratio,v_count_ratio,s_uniq_ratio,v_uniq_ratio,c_count_ratio,w_count_ratio,c_uniq_ratio,w_uniq_ratio,steps_ratio,S_steps_ratio,day_ratio,hours_ratio,time_ratio,p_freq[i]/p_freq[i+1],c_freq[i]/c_freq[i+1],p_density,c_density,loc_day,koef_weekday_1,koef_weekday_2,koef_weekday_3,koef_day_1,koef_day_2,koef_day_3,koef_week_mean,end_wrong,end_correct
0,4,0.0,0.0,1,1,0,1,0.0,0.0,0,0,4.0,0.4,0.0,0.516398,1.0,0.0,1,-1000.0,0.0,1.0,0.0,3.0,0.0,1.000000,0.0,4.836329,0.000000,7535.0,12874.0,0.0,0.0,0.000000,0.000000,0.250000,0.250000,0.000000,0.250000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,1.000000,0.020408,0.498662,4.836329,10.900931,0.296443,0.506491,0.050123,0.145941,0.151336,0.161449,0.033050,0.032876,0.032771,0.019739,0.0,0.0
1,6,0.0,0.0,1,1,0,1,0.0,0.0,0,0,4.0,0.4,0.0,0.516398,1.0,0.0,1,-1000.0,0.0,1.0,0.0,3.0,0.0,1.000000,0.0,4.836329,0.000000,7535.0,12874.0,0.0,0.0,0.000000,0.000000,0.250000,0.250000,0.000000,0.250000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,1.000000,0.020408,0.498662,4.836329,10.900931,0.296443,0.506491,0.050123,0.145941,0.151336,0.161449,0.031902,0.030232,0.031275,0.014451,0.0,0.0
2,10,0.0,0.0,2,2,0,2,0.0,0.0,0,4,12.0,1.2,0.0,1.932184,6.0,0.0,3,-1000.0,180.0,1.0,0.0,10.0,0.0,2.033005,0.0,4.492797,0.000000,1558.0,12874.0,0.0,0.0,0.000000,2.000000,0.166667,0.166667,0.000000,0.500000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,1.016503,1.0,1.000000,0.061224,0.498662,2.209929,10.900931,0.061295,0.506491,0.050123,0.118389,0.130860,0.145941,0.033050,0.032876,0.032771,0.016108,0.0,0.0
3,12,1.0,0.0,11,9,4,11,0.0,0.0,0,3,55.0,5.5,4.0,5.275731,14.0,0.0,1,0.0,2.0,1.0,1.0,39.0,0.0,10.999043,1.0,10.370013,0.253270,1154.0,1181.0,0.0,0.0,0.000000,0.272727,0.200000,0.163636,0.072727,0.254545,0.072727,0.200000,0.018182,0.000000,0.018182,0.000000,1.222116,1.0,1.000000,0.020408,0.418798,0.942810,0.253270,0.045401,0.046463,0.169160,0.161449,0.151801,0.140223,0.032876,0.032771,0.032528,0.025119,0.0,1.0
4,13,29.0,10.0,70,70,29,70,0.0,26.0,6,35,483.0,48.3,35.5,28.944583,105.0,10.0,15,0.0,0.0,2.0,2.0,240.0,105.0,70.000000,29.0,156.153846,146.318182,29.0,111.0,0.0,2.6,0.206897,0.500000,0.144928,0.144928,0.072464,0.217391,0.060041,0.144928,0.060041,0.074534,0.060041,0.020704,1.000000,1.0,0.695652,0.306122,0.418798,2.230769,5.045455,0.001141,0.004367,0.516052,0.151336,0.161449,0.151801,0.032622,0.033467,0.034963,0.015500,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6179,26791,0.0,0.0,1,1,0,1,0.0,0.0,0,0,4.0,0.4,0.0,0.516398,1.0,0.0,1,-1000.0,0.0,1.0,0.0,0.0,3.0,1.000000,0.0,4.836329,0.000000,7535.0,12874.0,0.0,0.0,0.000000,0.000000,0.250000,0.250000,0.000000,0.250000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,0.000000,0.020408,0.498662,4.836329,10.900931,0.296443,0.506491,0.050123,0.151801,0.140223,0.118389,0.031171,0.031902,0.030232,0.023478,0.0,0.0
6180,26795,0.0,0.0,1,1,0,1,0.0,0.0,0,0,4.0,0.4,0.0,0.516398,1.0,0.0,1,-1000.0,77.0,1.0,0.0,3.0,0.0,1.016974,0.0,4.918420,0.000000,7535.0,12874.0,0.0,0.0,0.000000,0.000000,0.250000,0.250000,0.000000,0.250000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,1.016974,1.0,1.000000,0.020408,0.498662,4.836329,10.900931,0.296443,0.506491,0.050123,0.151336,0.161449,0.151801,0.031554,0.029710,0.031171,0.019003,0.0,0.0
6181,26796,2.0,1.0,6,4,2,6,0.0,2.0,0,6,40.0,4.0,2.5,3.299832,12.0,1.0,2,17.0,0.0,1.0,1.0,29.

In [29]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6184 entries, 0 to 6183
Data columns (total 63 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   user_id                                         6184 non-null   int64  
 1   c_uniq                                          6184 non-null   float64
 2   w_uniq                                          6184 non-null   float64
 3   d_count                                         6184 non-null   int64  
 4   p_count                                         6184 non-null   int64  
 5   s_uniq                                          6184 non-null   int64  
 6   v_uniq                                          6184 non-null   int64  
 7   c_diff                                          6184 non-null   float64
 8   w_diff                                          6184 non-null   float64
 9   s_diff                                   